# Categorical Classification

# Importing Libraries and Loading data

In [5]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier, ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
# from google.colab import drive
import pandas as pd
import re
import spacy
from tqdm import tqdm
import time
import nltk
from nltk.corpus import stopwords
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    roc_auc_score,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    log_loss,
    confusion_matrix,
    classification_report
)
from tqdm import tqdm
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import GridSearchCV
import pickle

# drive.mount('/content/drive')
nlp = spacy.load('en_core_web_sm')

In [2]:
train = pd.read_csv("Data/original_data/train.csv", low_memory=False)
print(train.shape)

test = pd.read_csv("Data/original_data/test.csv", low_memory=False)
print(test.shape)

(93686, 3)
(31229, 3)


## Data Processing

### Duplicates removal

In [ ]:
train = train.drop_duplicates('crimeaditionalinfo')
print(train.shape)

train = train[~train['crimeaditionalinfo'].isnull()].reset_index(drop=True)
print(train.shape)

test = test[~test['crimeaditionalinfo'].isnull()].reset_index(drop=True)
print(test.shape)

(85014, 3)
(85013, 3)
(31222, 3)


## Data Pipelines

### SLL, SRLL, SRULL

In [ ]:
def do_correction(text):
    '''
    Correct misspelled words or hinglish words to English words
    '''
    changing_words = {'hai': 'is', 'ki': 'of', 'se': 'from', 'received': 'receive', 'ke': 'of', 'mere': 'my', 'started': 'start', 'ka': 'of',
    'dont': 'do not', 'rupees': 'rs', 'ko': 'to', 'aur': 'and', 'kiya': 'did', 'mujhe': 'me', 'par': 'but', 'liye': 'for', 'ek': 'one',
    'utr': 'answer', 'karta': 'does', 'maine': 'i', 'kr': 'do', 'isnt': 'is not', 'ho': 'yes', 'bhi': 'too', 'fir': 'then', 'mera': 'my', 'cant': 'can not',
    'aaya': 'came', 'paise': 'money', 'bola': 'said', 'diya': 'gave', 'im': 'i am', 'nhi': 'no', 'froud': 'fraud', 'ive': 'i have', 'nahi': 'no', 'liya': 'took',
    'ye': 'these', 'meri': 'my', 'plz': 'please', 'aap': 'you', 'gya': 'went', 'mene': 'i have', 'rupee': 'rs', 'hu': 'am', 'uske': 'his', 'jo': 'who', 'pls': 'please',
    'aadhar': 'aadhaar', 'karne': 'to do', 'gaya': 'went', 'wasnt': 'was not', 'inr': 'rs', 'wo': 'they', 'baad': 'after', 'rahe': 'are', 'de': 'give',
    'u': 'you', 'koi': 'any', 'paisa': 'rs', 'hua': 'happened', 'january': 'jan', 'sath': 'with', 'anydesk': 'any desk', 'nd': 'and', 'baroda': 'bob', 'frauder': 'fraud',
    'pese': 'rs', 'thats': 'that is', 'aya': 'came', 'kuch': 'some', 'krta': 'does', 'gye': 'went', 'karke': 'by doing', 'rhe': 'are', 'unhone': 'they', 'kaha': 'said',
    'na': 'no', 'bataya': 'told', 'fb': 'facebook', 'bol': 'tell', 'naam': 'names', 'krne': 'to do', 'unke': 'their', 'ab': 'now', 'phir': 'then', 'thi': 'was',
    'baat': 'talk', 'log': 'people', 'ref': 'reference', 'muje': 'me', 'oneoff': 'one off', 'rupey': 'rs', 'hadnt': 'had not', 'phonepay': 'phonepe', 'din': 'day',
    'everyones': 'everyone', 'relationships': 'relationship', 'diye': 'given', 'runaround': 'run around', 'kisi': 'any', 'vo': 'they', 'rupay': 'rs',
    'apne': 'our', 'ltd': 'limited', 'btaya': 'told', 'le': 'take', 'uska': 'his', 'aa': 'come', 'gaye': 'went', 'frod': 'fraud', 'february': 'feb', 'bhej': 'send',
    'hain': 'are', 'mein': 'in', 'dhamki': 'threat', 'rupe': 'rs', 'frouder': 'fraud', 'kare': 'do', 'bahut': 'very', 'rupaye': 'rs', 'december': 'dec',
    'apna': 'own', 'jab': 'when', 'ie': 'that is', 'iam': 'i am', 'sirmadam': 'sir madam', 'lekin': 'but', 'dene': 'to give', 'messaged': 'messaged',
    'abhi': 'now', 'mujhse': 'from me', 'aapko': 'to you', 'vedio': 'video', 'bat': 'talk', 'pas': 'near', 'ammount': 'amount', 'creditdebit': 'credit debit',
    'ha': 'yes', 'lo': 'take', 'toh': 'so', 'karna': 'to do', 'ur': 'your', 'didnot': 'did not', 'sab': 'all', 'rahi': 'doing', 'karo': 'do it', 'gali': 'street',
    'inke': 'their', 'pvt': 'private', 'holdreverse': 'hold reverse', 'jis': 'that', 'hoga': 'will be', 'kiye': 'did', 'dena': 'give', 'apko': 'to you',
    'watsapp': 'whatsapp', 'krke': 'by doing', 'nikal': 'remove', 'asap': 'as soon as possible', 'karte': 'do', 'usko': 'for him', 'suspecious': 'suspicious',
    'tak': 'till', 'jayega': 'will go', 'bheja': 'sent', 'peoples': 'people', 'unathorised': 'unauthorised', 'otps': 'otp', 'hackd': 'hacked', 'ap': 'you',
    'jisme': 'in which', 'hrs': 'hours', 'dwara': 'by', 'laga': 'put', 'ji': 'yes', 'fruad': 'fraud', 'bta': 'tell me', 'jaye': 'go', 'aapke': 'your',
    'aaj': 'today', 'pata': 'know', 'kijiye': 'do it', 'ni': 'no', 'jiska': 'whose', 'pornographic': 'pornography', 'fruader': 'fraud', 'fruadster': 'fraud',
    'fruads': 'fraud', 'fruaded': 'fraud', 'gpayphone': 'gpay phone', 'phonepegpay': 'phonepe gpay', 'fruade': 'fraud', 'fruadent': 'fraud', 'paytmgpay': 'paytm gpay',
    'gpayphonepay': 'gpay phonepe', 'pornograpy': 'pornography', 'fruadest': 'fraud', 'fruadsters': 'fraud', 'fruading': 'fraud', 'cimplaint': 'complaint',
    'gpaypaytm': 'gpay paytm', 'gpayeuronettotal': 'gpay euronettotal', 'phonepaygpay': 'phonepe gpay', 'pornogragy': 'pornography', 'childporns': 'child porn',
    'gpayat': 'gpay at', 'gpayhe': 'is gpay', 'govtsalary': 'government salary', 'gpaymy': 'gpay my', 'gpayphonepe': 'gpay phonepe', 'bygpay': 'by gpay',
    'gpayed': 'gpay', 'gpaybut': 'gpay but', 'gpaythey': 'gpay they', 'afruad': 'a fraud', 'fruadulant': 'fraud', 'pornograpgy': 'pornography',
    'gpaythen': 'gpay then', 'fruadulent': 'fraud', 'hellomy': 'hello my', 'bankingpay': 'banking pay', 'gpayinitial': 'gpay initial', 'gpaylater': 'gpay later',
    'whatappgpay': 'whatsapp gpay', 'fruadamount': 'fraud amount', 'lostvictim': 'lost victim', 'padhega': 'will read', 'varificaon': 'verification',
    'maiupulating': 'manipulating', 'nowadys': 'now a days', 'hasband': 'husband', 'subcribers': 'subscriber', 'youtubewhen': 'youtube when', 'instarction': 'interaction',
    'cardupimobile': 'card upi mobile', 'rejecied': 'rejected', 'bhahen': 'sister', 'comingthey': 'coming they', 'amountfirsy': 'amount first', 'uhave': 'you have',
    'roadowner': 'road owner', 'readable': 'read able', 'paasport': 'passport', 'latere': 'later', 'tryinh': 'trying', 'gpayon': 'gpay on', 'praveengpay': 'praveen gpay',
    'upipaytmgpay': 'upi paytm gpay', 'gpaylink': 'gpay link', 'gpayamount': 'gpay amount', 'intrnet': "internet", "secreate": "secret", "victime": "victim",
    "apka": "your", "apke": "your", 'aapka': 'yours', 'abusive': 'abuse', 'acc': 'account', 'accidentally': 'accident', 'ad': 'advertisement', 'adhar': 'aadhaar', 'alag': 'apart',
    'amt': 'amount', 'apka': 'your', 'apke': 'your', 'app': 'application', 'approval': 'approve', 'attachment': 'attach', 'baar': 'times', 'band': 'close',
    'bankar': 'become', 'banking': 'bank', 'bar': 'times', 'bata': 'tell', 'blackmailing': 'blackmail', 'booking': 'book', 'careful': 'care',
    'chala': 'walked', 'chesaru': 'done', 'clothe': 'cloth', 'complainant': 'complaint', 'completely': 'complete', 'confirmation': 'confirm',
    'constantly': 'constant', 'continuously': 'continuous', 'dal': 'did', 'deduction': 'deduct', 'delivery': 'deliver', 'denge': 'give',
    'dhani': 'rich', 'di': 'gave', 'dijiye': 'give', 'directly': 'direct', 'dist': 'district', 'dunga': 'give', 'firstly': 'first', 'fraudster': 'fraud',
    'fraudulent': 'fraud', 'frezzing': 'freeze', 'froder': 'fraud', 'frustrated': 'frustrate', 'galat': 'wrong', 'gandi': 'dirty', 'getting': 'get',
    'googlepay': 'gpay', 'harassment': 'harass', 'harrasse': 'harass', 'holder': 'hold', 'hone': 'to', 'honestly': 'honest', 'hum': 'we',
    'immediately': 'immediate', 'impersonation': 'impersonate', 'informed': 'inform', 'initially': 'initial', 'insta': 'instagram', 'instal': 'install',
    'instruction': 'instruct', 'investigation': 'investigate', 'isne': 'has', 'ja': 'go', 'jankar': 'knowing', 'jankari': 'information',
    'jiske': 'whose', 'kar': 'do', 'karwa': 'did', 'kat': 'cut', 'kra': 'do', 'kro': 'do', 'kya': 'what', 'lac': 'lakh', 'ladki': 'lady', 'lal': 'red',
    'li': 'took', 'log': 'people', 'lone': 'loan', 'mai': 'i', 'mang': 'demand', 'mentally': 'mental', 'mila': 'found', 'min': 'minimum', 'mo': 'mobile',
    'mob': 'mobile', 'msg': 'message', 'msgs': 'message', 'nam': 'name', 'ne': 'near', 'nos': 'number', 'notifiction': 'notification', 'num': 'number',
    'pahle': 'first', 'pareshan': 'worried', 'pe': 'pay', 'pende': 'pending', 'ph': 'phone', 'pic': 'picture', 'pr': 'but', 'pta': 'know', 'raha': 'remained',
    'recieve': 'receive', 'recive': 'receive', 'recording': 'record', 'reference': 'reference', 'refrence': 'reference', 'refunded': 'refund', 'rha': 'remained',
    'rhi': 'doing', 'sare': 'ok', 'saving': 'save', 'scamme': 'scammer', 'smishe': 'funny', 'st': 'street', 'stange': 'strange', 'suchana': 'information',
    'suchna': 'information', 'suchnakrta': 'inform ', 'terrifying': 'terrify', 'texte': 'text', 'thana': 'station', 'threate': 'threat', 'trans': 'transaction',
    'transacation': 'transaction', 'transaction': 'transaction', 'transction': 'transaction', 'transection': 'transaction', 'usme': 'in that', 'usne': 'he',
    'vishe': 'special', 'wale': 'ones', 'wapas': 'back', 'week': 'weak', 'whatsup': 'whatsapp', 'withdrawal': 'withdraw', 'working': 'work', 'ya': 'yes',
    'hun': 'am', 'nahin': 'no', 'agar': 'if', 'tha': 'was', 'aayi': 'came', 'iske': 'it is', 'unka': 'their', 'dete': 'give', 'kre': 'do',
    'donot':'do not', 'lekar': 'taking', 'march': 'mar', 'january': 'jan', 'february': 'feb', 'april': 'apr', 'june': 'jun', 'july': 'jul',
    'august': 'aug','september':'sept', 'october': 'oct','november': 'nov', 'december': 'dec', 'aisa': 'as such'
    }

    # Correct the words
    corrected_msg = ''
    all_keys = changing_words.keys()
    for word in text.split():
        if word in all_keys:
            corrected_msg += changing_words[word]
        else:
            corrected_msg += word
        corrected_msg += " "
    return corrected_msg

def do_sensitive_website_replacement(corrected_msg, website_match_findings):
  '''
  Replacing sensitive websites with the sensitive word that is matched
  '''
  sensitive_keywords = [
        "porn", "xxx", "hentai", "xvideo", "adult", "breast","pussy", "dick", "cock","sex", "erotic", "nude", "camgirl", "cams", "cheat",
        "escort", "fetish", "bdsm", "taboo", "incest", "shemale", "milf", "barelylegal", "amateur",
        "anal", "fetish", "hotgirls", "kinky", "hardcore", "lingerie", "webcam", "strip", "nsfw",
        "onlyfans", "redtube", "youjizz", "spankbang", "tnaflix", "xtube", "nudity", "playboy",
        "bangbros", "brazzers", "naughty", "mature", "teens", "voyeur", "threesome", "gangbang",
        "peeping", "exposed", "babes", "slut", "masturbate", "hookup", "sugarbaby", "paypig", "comic"
  ]

  new_msg = ''
  for word in corrected_msg.strip().split():
    # print(word)
    if len(website_match_findings) > 0:
        if word in website_match_findings and 'express' not in word and 'gmail' not in word and 'internet' not in word:
            # Check if it has sensitive word
            for sen_word in sensitive_keywords:
                if sen_word in word:
                    new_msg += sen_word
        else:
          new_msg += word
    else:
        new_msg += word
    new_msg += " "
  return new_msg

In [ ]:
## DATA PROCESSING
'''
1. stopwords
2. lower
3. replace mispelled words
4. detect websites and detect sensitive words and replace them, else remove
5. lemma
'''
stop_words = [
    "0o", "0s", "3a", "3b", "3d", "6b", "6o", "a", "a1", "a2", "a3", "a4", "ab", "able", "about", "above", "abst", "ac", "accordance", "according",
    "accordingly", "across", "act", "actually", "ad", "added", "adj", "ae", "af", "affected", "affecting", "affects", "after", "afterwards", "ag",
    "again", "against", "ah", "ain", "ain't", "aj", "al", "all", "allow", "allows", "almost", "alone", "along", "already", "also", "although",
    "always", "am", "among", "amongst", "amoungst", "amount", "an", "and", "announce", "another", "any", "anybody", "anyhow", "anymore", "anyone",
    "anything", "anyway", "anyways", "anywhere", "ao", "ap", "apart", "apparently", "appear", "appreciate", "appropriate", "approximately", "ar",
    "are", "aren", "arent", "aren't", "arise", "around", "as", "a's", "aside", "ask", "asking", "associated", "at", "au", "auth", "av", "available",
    "aw", "away", "awfully", "ax", "ay", "az", "b", "b1", "b2", "b3", "ba", "back", "bc", "bd", "be", "became", "because", "become", "becomes",
    "becoming", "been", "before", "beforehand", "begin", "beginning", "beginnings", "begins", "behind", "being", "believe", "below", "beside",
    "besides", "best", "better", "between", "beyond", "bi", "bill", "biol", "bj", "bk", "bl", "bn", "both", "bottom", "bp", "br", "brief",
    "briefly", "bs", "bt", "bu", "but", "bx", "by", "c", "c1", "c2", "c3", "ca", "call", "came", "can", "cannot", "cant", "can't", "cause",
    "causes", "cc", "cd", "ce", "certain", "certainly", "cf", "cg", "ch", "changes", "ci", "cit", "cj", "cl", "clearly", "cm", "c'mon", "cn", "co",
    "com", "come", "comes", "con", "concerning", "consequently", "consider", "considering", "contain", "containing", "contains", "corresponding",
    "could", "couldn", "couldnt", "couldn't", "course", "cp", "cq", "cr", "cry", "cs", "c's", "ct", "cu", "currently", "cv", "cx", "cy", "cz", "d",
    "d2", "da", "date", "dc", "dd", "de", "definitely", "describe", "described", "despite", "detail", "df", "di", "did", "didn", "didn't", "different",
    "dj", "dk", "dl", "do", "does", "doesn", "doesn't", "doing", "don", "done", "don't", "down", "downwards", "dp", "dr", "ds", "dt", "du", "due",
    "during", "dx", "dy", "e", "e2", "e3", "ea", "each", "ec", "ed", "edu", "ee", "ef", "effect", "eg", "ei", "eight", "eighty", "either", "ej", "el",
    "eleven", "else", "elsewhere", "em", "empty", "en", "end", "ending", "enough", "entirely", "eo", "ep", "eq", "er", "es", "especially", "est", "et",
    "et-al", "etc", "eu", "ev", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex", "exactly", "example", "except", "ey",
    "f", "f2", "fa", "far", "fc", "few", "ff", "fi", "fifteen", "fifth", "fify", "fill", "find", "fire", "first", "five", "fix", "fj", "fl", "fn", "fo",
    "followed", "following", "follows", "for", "former", "formerly", "forth", "forty", "found", "four", "fr", "from", "front", "fs", "ft", "fu", "full",
    "further", "furthermore", "fy", "g", "ga", "gave", "ge", "get", "gets", "getting", "gi", "give", "given", "gives", "giving", "gj", "gl", "go", "goes",
    "going", "gone", "got", "gotten", "gr", "greetings", "gs", "gy", "h", "h2", "h3", "had", "hadn", "hadn't", "happens", "hardly", "has", "hasn", "hasnt",
    "hasn't", "have", "haven", "haven't", "having", "he", "hed", "he'd", "he'll", "hello", "help", "hence", "her", "here", "hereafter", "hereby", "herein",
    "heres", "here's", "hereupon", "hers", "herself", "hes", "he's", "hh", "hi", "hid", "him", "himself", "his", "hither", "hj", "ho", "home", "hopefully",
    "how", "howbeit", "however", "how's", "hr", "hs", "http", "hu", "hundred", "hy", "i", "i2", "i3", "i4", "i6", "i7", "i8", "ia", "ib", "ibid", "ic", "id",
    "i'd", "ie", "if", "ig", "ignored", "ih", "ii", "ij", "il", "i'll", "im", "i'm", "immediate", "immediately", "importance", "important", "in", "inasmuch",
    "inc", "indeed", "index", "indicate", "indicated", "indicates", "information", "inner", "insofar", "instead", "interest", "into", "invention", "inward",
    "io", "ip", "iq", "ir", "is", "isn", "isn't", "it", "itd", "it'd", "it'll", "its", "it's", "itself", "iv", "i've", "ix", "iy", "iz", "j", "jj", "jr",
    "js", "jt", "ju", "just", "k", "ke", "keep", "keeps", "kept", "kg", "kj", "km", "know", "known", "knows", "ko", "l", "l2", "la", "largely", "last",
    "lately", "later", "latter", "latterly", "lb", "lc", "le", "least", "les", "less", "lest", "let", "lets", "let's", "lf", "like", "liked", "likely",
    "line", "little", "lj", "ll", "ll", "ln", "lo", "look", "looking", "looks", "los", "lr", "ls", "lt", "ltd", "m", "m2", "ma", "made", "mainly", "make",
    "makes", "many", "may", "maybe", "me", "mean", "means", "meantime", "meanwhile", "merely", "mg", "might", "mightn", "mightn't", "mill", "million",
    "mine", "miss", "ml", "mn", "mo", "more", "moreover", "most", "mostly", "move", "mr", "mrs", "ms", "mt", "mu", "much", "mug", "must", "mustn", "mustn't",
    "my", "myself", "n", "n2", "na", "name", "namely", "nay", "nc", "nd", "ne", "near", "nearly", "necessarily", "necessary", "need", "needn", "needn't",
    "needs", "neither", "never", "nevertheless", "new", "next", "ng", "ni", "nine", "ninety", "nj", "nl", "nn", "no", "nobody", "non", "none", "nonetheless",
    "noone", "nor", "normally", "nos", "not", "noted", "nothing", "novel", "now", "nowhere", "nr", "ns", "nt", "ny", "o", "oa", "ob", "obtain", "obtained",
    "obviously", "oc", "od", "of", "off", "often", "og", "oh", "oi", "oj", "ok", "okay", "ol", "old", "om", "omitted", "on", "once", "one", "ones", "only",
    "onto", "oo", "op", "oq", "or", "ord", "os", "ot", "other", "others", "otherwise", "ou", "ought", "our", "ours", "ourselves", "out", "outside", "over",
    "overall", "ow", "owing", "own", "ox", "oz", "p", "p1", "p2", "p3", "page", "pagecount", "pages", "par", "part", "particular", "particularly", "pas",
    "past", "pc", "pd", "pe", "per", "perhaps", "pf", "ph", "pi", "pj", "pk", "pl", "placed", "please", "plus", "pm", "pn", "po", "poorly", "possible",
    "possibly", "potentially", "pp", "pq", "pr", "predominantly", "present", "presumably", "previously", "primarily", "probably", "promptly", "proud",
    "provides", "ps", "pt", "pu", "put", "py", "q", "qj", "qu", "que", "quickly", "quite", "qv", "r", "r2", "ra", "ran", "rather", "rc", "rd", "re",
    "readily", "really", "reasonably", "recent", "recently", "ref", "refs", "regarding", "regardless", "regards", "related", "relatively", "research",
    "research-articl", "respectively", "resulted", "resulting", "results", "rf", "rh", "ri", "right", "rj", "rl", "rm", "rn", "ro", "rq", "rr", "rs", "rt",
    "ru", "run", "rv", "ry", "s", "s2", "sa", "said", "same", "saw", "say", "saying", "says", "sc", "sd", "se", "sec", "second", "secondly", "section",
    "see", "seeing", "seem", "seemed", "seeming", "seems", "seen", "self", "selves", "sensible", "sent", "serious", "seriously", "seven", "several", "sf",
    "shall", "shan", "shan't", "she", "shed", "she'd", "she'll", "shes", "she's", "should", "shouldn", "shouldn't", "should've", "show", "showed", "shown",
    "showns", "shows", "si", "side", "significant", "significantly", "similar", "similarly", "since", "sincere", "six", "sixty", "sj", "sl", "slightly", "sm",
    "sn", "so", "some", "somebody", "somehow", "someone", "somethan", "something", "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "sp",
    "specifically", "specified", "specify", "specifying", "sq", "sr", "ss", "st", "still", "stop", "strongly", "sub", "substantially", "successfully", "such",
    "sufficiently", "suggest", "sup", "sure", "sy", "system", "sz", "t", "t1", "t2", "t3", "take", "taken", "taking", "tb", "tc", "td", "te", "tell", "ten",
    "tends", "tf", "th", "than", "thank", "thanks", "thanx", "that", "that'll", "thats", "that's", "that've", "the", "their", "theirs", "them", "themselves",
    "then", "thence", "there", "thereafter", "thereby", "thered", "therefore", "therein", "there'll", "thereof", "therere", "theres", "there's", "thereto",
    "thereupon", "there've", "these", "they", "theyd", "they'd", "they'll", "theyre", "they're", "they've", "thickv", "thin", "think", "third", "this",
    "thorough", "thoroughly", "those", "thou", "though", "thoughh", "thousand", "three", "throug", "through", "throughout", "thru", "thus", "ti", "til",
    "tip", "tj", "tl", "tm", "tn", "to", "together", "too", "took", "top", "toward", "towards", "tp", "tq", "tr", "tried", "tries", "truly", "try", "trying",
    "ts", "t's", "tt", "tv", "twelve", "twenty", "twice", "two", "tx", "u", "u201d", "ue", "ui", "uj", "uk", "um", "un", "under", "unfortunately",
    "unless", "unlike", "unlikely", "until", "unto", "uo", "up", "upon", "ups", "ur", "us", "use", "used", "useful", "usefully", "usefulness", "uses",
    "using", "usually", "ut", "v", "va", "value", "various", "vd", "ve", "ve", "very", "via", "viz", "vj", "vo", "vol", "vols", "volumtype", "vq", "vs",
    "vt", "vu", "w", "wa", "want", "wants", "was", "wasn", "wasnt", "wasn't", "way", "we", "wed", "we'd", "welcome", "well", "we'll", "well-b", "went",
    "were", "we're", "weren", "werent", "weren't", "we've", "what", "whatever", "what'll", "whats", "what's", "when", "whence", "whenever", "when's",
    "where", "whereafter", "whereas", "whereby", "wherein", "wheres", "where's", "whereupon", "wherever", "whether", "which", "while", "whim", "whither",
    "who", "whod", "whoever", "whole", "who'll", "whom", "whomever", "whos", "who's", "whose", "why", "why's", "wi", "widely", "will", "willing", "wish",
    "with", "within", "without", "wo", "won", "wonder", "wont", "won't", "words", "world", "would", "wouldn", "wouldnt", "wouldn't",
    "xt", "xv", "y", "y2", "yes", "yet", "yj", "yl", "you", "youd", "you'd", "you'll", "your", "youre", "you're", "yours", "yourself",
    "yourselves", "you've", "yr", "ys", "yt", "z", "zero", "zi", "zz", 'ag', 'amit', 'ani', 'dec', 'delhi', 'devi', 'feb', 'five', 'gupta', 'hi', 'hui',
    'jaipur', 'jan', 'kumar', 'may', 'rahul', 'ram', 'sharma', 'singh', 'yadav', 'mar', 'apr', 'jun', 'jul', 'aug', 'sept', 'oct', 'nov']

train_SRULL = []
train_SLL = []
train_SRLL = []

for msg in tqdm(train['crimeaditionalinfo'].tolist(), desc = 'processing train:'):
  # SLL
  text = msg.lower().strip()
  tokens = text.split()
  tokens = [word for word in tokens if word not in stop_words]
  # Lemmatization using spacy
  doc = nlp(' '.join(tokens))
  lemmatized_tokens = [token.lemma_ for token in doc]
  train_SLL.append(' '.join(lemmatized_tokens).strip())

  # SRLL
  text = msg.lower().strip()
  corrected_msg = do_correction(text)
  tokens = corrected_msg.split()
  tokens = [word for word in tokens if word not in stop_words]
  lemmatized_tokens = [token.lemma_ for token in doc]
  train_SRLL.append(' '.join(lemmatized_tokens).strip())

  # SRULL
  text = msg.lower().strip()
  corrected_msg = do_correction(text)

  doubt_website_pattern = r'\b(?:http|https|www)[a-zA-Z0-9]+(?:dot|at)?[a-zA-Z0-9]+(?:com|org|net)\b|\b[a-zA-Z0-9._%+-]+(?:at|dot)?[a-zA-Z0-9]+(?:com|org|net)\b'
  website_match_findings = re.findall(doubt_website_pattern, corrected_msg)
  new_msg = do_sensitive_website_replacement(corrected_msg, website_match_findings)

  tokens = new_msg.split()
  tokens = [word for word in tokens if word not in stop_words]
  lemmatized_tokens = [token.lemma_ for token in doc]
  train_SRULL.append(' '.join(lemmatized_tokens).strip())

train['SRULL_msg'] = train_SRULL
train['SLL_msg'] = train_SLL
train['SRLL_msg'] = train_SRLL

processing train:: 100%|██████████| 85013/85013 [27:33<00:00, 51.40it/s]


Train saved


In [ ]:
test_SLL = []
test_SRLL = []
test_SRULL = []

for msg in tqdm(test['crimeaditionalinfo'], desc = 'processing test:'):
  # SLL
  text = msg.lower().strip()
  tokens = text.split()
  tokens = [word for word in tokens if word not in stop_words]
  # Lemmatization using spacy
  doc = nlp(' '.join(tokens))
  lemmatized_tokens = [token.lemma_ for token in doc]
  test_SLL.append(' '.join(lemmatized_tokens).strip())

  # SRLL
  text = msg.lower().strip()
  corrected_msg = do_correction(text)
  tokens = corrected_msg.split()
  tokens = [word for word in tokens if word not in stop_words]
  lemmatized_tokens = [token.lemma_ for token in doc]
  test_SRLL.append(' '.join(lemmatized_tokens).strip())

  # SRULL
  text = msg.lower().strip()
  corrected_msg = do_correction(text)

  doubt_website_pattern = r'\b(?:http|https|www)[a-zA-Z0-9]+(?:dot|at)?[a-zA-Z0-9]+(?:com|org|net)\b|\b[a-zA-Z0-9._%+-]+(?:at|dot)?[a-zA-Z0-9]+(?:com|org|net)\b'
  website_match_findings = re.findall(doubt_website_pattern, corrected_msg)
  new_msg = do_sensitive_website_replacement(corrected_msg, website_match_findings)

  tokens = new_msg.split()
  tokens = [word for word in tokens if word not in stop_words]
  lemmatized_tokens = [token.lemma_ for token in doc]
  test_SRULL.append(' '.join(lemmatized_tokens).strip())

test['SLL_msg'] = test_SLL
test['SRLL_msg'] = test_SRLL
test['SRULL_msg'] = test_SRULL

processing test:: 100%|██████████| 31222/31222 [09:41<00:00, 53.71it/s]


Test saved


In [ ]:
train.to_csv("Data/processed_data/final_train.csv",index=False)
print("Train saved")

test.to_csv("Data/processed_data/final_test.csv",index=False)
print("Test saved")

Train saved
Test saved


## Category grouping

**Label: 0 (Women/Child Related Crime)**
1. Sexually Obscene material
2. Sexually Explicit Act
3. Child Pornography CPChild Sexual Abuse Material CSAM
4. RapeGang Rape RGRSexually Abusive Content

**Label: 1 (Financial Crime)**
1. Online Financial Fraud
2. Cryptocurrency Crime
3. Online Gambling  Betting

**Label: 2 (Other Cyber Crime)**
1. Online and Social Media Related Crime
2. Any Other Cyber Crime
3. Cyber Attack/ Dependent Crimes
4. Hacking  Damage to computercomputer system etc
5. Cyber Terrorism
6. Ransomware
7. Online Cyber Trafficking
8. Report Unlawful Content

In [6]:
train = pd.read_csv("Data/processed_data/final_train.csv", low_memory=False)
print(train.shape)

test = pd.read_csv("Data/processed_data/final_test.csv", low_memory=False)
print(test.shape)

(84040, 13)
(28328, 13)


In [ ]:
def create_final_category(val):
  if val in ['Sexually Obscene material', 'Sexually Explicit Act', 'Child Pornography CPChild Sexual Abuse Material CSAM',
 'RapeGang Rape RGRSexually Abusive Content']:
    return 0
  elif val in ['Online Financial Fraud', 'Cryptocurrency Crime', 'Online Gambling  Betting']:
    return 1
  elif val in ['Online and Social Media Related Crime',
 'Any Other Cyber Crime',
 'Cyber Attack/ Dependent Crimes',
 'Hacking  Damage to computercomputer system etc',
 'Cyber Terrorism',
 'Ransomware', 'Report Unlawful Content', 'Online Cyber Trafficking']:
    return 2
  else:
    return 3

train['final_category'] = train['category'].apply(create_final_category)
test['final_category'] = test['category'].apply(create_final_category)
print(train.shape, test.shape)

train = train[~train['final_category'].eq(3)].reset_index(drop=True)
test = test[~test['final_category'].eq(3)].reset_index(drop=True)
print(train.shape, test.shape)

(85013, 7) (31218, 7)
(85013, 7) (31218, 7)


# Category Classification

We experimented by building **BoW** and **Tf-IDF** embeddings over **SLL, SRLL, SRULL** processed data and trained on different models - **Logistic Regression, Naive Bayes, Random Forest, CatBoost, Gradient Boosting.** We finalised three pipelines which provides better performance and stability over train and OOT. They are:

1. NB-SRULL-TFIDF: We use SRULL processed data with TF-IDF embedding with max_features = 1000 and classify using Naive Bayes.

2. LR-SRLL-TFIDF: We use SRLL processed data with TF-IDF embedding with max_features = 1000 and classify using Logistic Regression.

3. LR-SRULL-TFIDF: We use SRULL processed data with TF-IDF embedding with max_features = 1000 and classify using Logistic Regression.

In [ ]:
def classwise_scores_and_overall(model, X_test_embed, y_test):

  y_prob = model.predict_proba(X_test_embed)
  y_pred = model.predict(X_test_embed)
  y_test_bin = label_binarize(y_test, classes=[0, 1, 2])

  # Calculate per-class metrics
  metrics = []
  for i, class_label in enumerate(model.classes_):
      class_auc = roc_auc_score(y_test_bin[:, i], y_prob[:, i])
      class_accuracy = accuracy_score((y_test == class_label).astype(int), (y_pred == class_label).astype(int))
      class_f1 = f1_score((y_test == class_label).astype(int), (y_pred == class_label).astype(int))
      class_recall = recall_score((y_test == class_label).astype(int), (y_pred == class_label).astype(int))
      class_precision = precision_score((y_test == class_label).astype(int), (y_pred == class_label).astype(int))
      class_loss = log_loss(y_test_bin[:, i], y_prob[:, i])

      metrics.append({
          'Class': class_label,
          'AUC': class_auc,
          'Accuracy': class_accuracy,
          'F1 Score': class_f1,
          'Recall': class_recall,
          'Precision': class_precision,
          'Log Loss': class_loss
      })

  metrics_df = pd.DataFrame(metrics)

  # Calculate overall metrics
  overall_accuracy = accuracy_score(y_test, y_pred)
  overall_f1 = f1_score(y_test, y_pred, average='weighted')
  overall_recall = recall_score(y_test, y_pred, average='weighted')
  overall_precision = precision_score(y_test, y_pred, average='weighted')
  overall_auc = roc_auc_score(y_test_bin, y_prob, average='weighted', multi_class='ovr')
  overall_loss = log_loss(y_test_bin, y_prob)

  # Display overall metrics
  overall_metrics = {
      'Overall Accuracy': overall_accuracy,
      'Overall F1 Score': overall_f1,
      'Overall Recall': overall_recall,
      'Overall Precision': overall_precision,
      'Overall AUC': overall_auc,
      'Overall Log Loss': overall_loss
  }
  overall_metrics_df = pd.DataFrame([overall_metrics])
  return metrics_df, overall_metrics_df

## NB-SRULL-TFIDF

In [ ]:
print(train.shape, test.shape)
train = train[~train['SRULL_msg'].isnull()].reset_index(drop=True)
test = test[~test['SRULL_msg'].isnull()].reset_index(drop=True)
print(train.shape, test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train['SRULL_msg'], train['final_category'], test_size=0.25, random_state=42, stratify=train['final_category'])
print(X_train.shape, X_test.shape)

# tfidf_vectorizer = TfidfVectorizer(max_features = 1000)
with open("Assets/CategoryClassification/NB_SRULL_TFIDF/tfidf_vectorizer.pkl", "rb") as f:
    tfidf_vectorizer = pickle.load(f)

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)
oot_tfidf = tfidf_vectorizer.transform(test['SRULL_msg'])
overall_tfidf = tfidf_vectorizer.transform(train['SRULL_msg'])

y_oot = test['final_category'].tolist()
y_train_overall = train['final_category'].tolist()

(84421, 7) (30791, 7)
(84421, 7) (30791, 7)
(63315,) (21106,)


In [ ]:
# model = MultinomialNB(alpha=5, fit_prior=True)
with open("Assets/CategoryClassification/NB_SRULL_TFIDF/naive_bayes.pkl", "rb") as f:
    model = pickle.load(f)

model.fit(X_train_tfidf, y_train)

train_class_wise, train_overall = classwise_scores_and_overall(model, X_train_tfidf, y_train)
test_class_wise, test_overall = classwise_scores_and_overall(model,X_test_tfidf, y_test)
oot_class_wise, oot_overall = classwise_scores_and_overall(model,oot_tfidf, y_oot)
train_overall_class_wise, train_overall_overall = classwise_scores_and_overall(model,overall_tfidf, y_train_overall)

classwise = pd.concat([train_class_wise, test_class_wise, train_overall_class_wise, oot_class_wise])
classwise.index = ['train_0', 'train_1', 'train_2', 'test_0', 'test_1', 'test_2',
                   'train_overall_0','train_overall_1','train_overall_2','oot_0', 'oot_1', 'oot_2']

overall = pd.concat([train_overall, test_overall, train_overall_overall, oot_overall])
overall.index = ['train', 'test', 'train_overall','oot']
overall.T

,train,test,train_overall,oot
Overall Accuracy,0.781600,0.783758,0.782140,0.763567
Overall F1 Score,0.765259,0.767189,0.765749,0.737903
Overall Recall,0.781600,0.783758,0.782140,0.763567
Overall Precision,0.768122,0.769679,0.768533,0.743689
Overall AUC,0.889339,0.889313,0.889335,0.896500
Overall Log Loss,0.484507,0.485958,0.484870,0.491247


## LR-SRLL-TFIDF

In [ ]:
print(train.shape, test.shape)
train = train[~train['SRLL_msg'].isnull()].reset_index(drop=True)
test = test[~test['SRLL_msg'].isnull()].reset_index(drop=True)
print(train.shape, test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train['SRLL_msg'], train['final_category'], test_size=0.25, random_state=42, stratify=train['final_category'])
print(X_train.shape, X_test.shape)

# tfidf_vectorizer = TfidfVectorizer(max_features = 1000)  # You can customize max_features
with open("Assets/CategoryClassification/LR_SRLL_TFIDF/tfidf_vectorizer.pkl", "rb") as f:
    tfidf_vectorizer = pickle.load(f)

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)
oot_tfidf = tfidf_vectorizer.transform(test['SRLL_msg'])
overall_tfidf = tfidf_vectorizer.transform(train['SRLL_msg'])

y_oot = test['final_category'].tolist()
y_train_overall = train['final_category'].tolist()

(84421, 7) (30791, 7)
(84421, 7) (30791, 7)
(63315,) (21106,)


In [ ]:
# model = LogisticRegression(C=0.1, class_weight=None, penalty='l2', solver='lbfgs')
with open("Assets/CategoryClassification/LR_SRLL_TFIDF/logistic_regression.pkl", "rb") as f:
    model = pickle.load(f)

model.fit(X_train_tfidf, y_train)

train_class_wise, train_overall = classwise_scores_and_overall(model, X_train_tfidf, y_train)
test_class_wise, test_overall = classwise_scores_and_overall(model,X_test_tfidf, y_test)
oot_class_wise, oot_overall = classwise_scores_and_overall(model,oot_tfidf, y_oot)
train_overall_class_wise, train_overall_overall = classwise_scores_and_overall(model,overall_tfidf, y_train_overall)

classwise = pd.concat([train_class_wise, test_class_wise, train_overall_class_wise, oot_class_wise])
classwise.index = ['train_0', 'train_1', 'train_2', 'test_0', 'test_1', 'test_2',
                   'train_overall_0','train_overall_1','train_overall_2','oot_0', 'oot_1', 'oot_2']

overall = pd.concat([train_overall, test_overall, train_overall_overall, oot_overall])
overall.index = ['train', 'test', 'train_overall','oot']
overall.T

,train,test,train_overall,oot
Overall Accuracy,0.799874,0.794608,0.798557,0.778961
Overall F1 Score,0.784507,0.780257,0.783444,0.753659
Overall Recall,0.799874,0.794608,0.798557,0.778961
Overall Precision,0.793094,0.788995,0.792064,0.772064
Overall AUC,0.906029,0.901644,0.904939,0.908307
Overall Log Loss,0.456988,0.465809,0.459193,0.485340


## LR-SRULL-TFIDF

In [ ]:
print(train.shape, test.shape)
train = train[~train['SRULL_msg'].isnull()].reset_index(drop=True)
test = test[~test['SRULL_msg'].isnull()].reset_index(drop=True)
print(train.shape, test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train['SRULL_msg'], train['final_category'], test_size=0.25, random_state=42, stratify=train['final_category'])
print(X_train.shape, X_test.shape)

# tfidf_vectorizer = TfidfVectorizer(max_features = 1000)  # You can customize max_features
with open("Assets/CategoryClassification/LR_SRULL_TFIDF/tfidf_vectorizer.pkl", "rb") as f:
    tfidf_vectorizer = pickle.load(f)

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)
oot_tfidf = tfidf_vectorizer.transform(test['SRULL_msg'])
overall_tfidf = tfidf_vectorizer.transform(train['SRULL_msg'])

y_oot = test['final_category'].tolist()
y_train_overall = train['final_category'].tolist()

(84421, 7) (30791, 7)
(84421, 7) (30791, 7)
(63315,) (21106,)


In [ ]:
# 'class_weight': None, 'max_iter': 500, 'solver': 'lbfgs'

# model = LogisticRegression(class_weight = None, max_iter = 500, solver = 'lbfgs')
with open("Assets/CategoryClassification/LR_SRULL_TFIDF/logistic_regression.pkl", "rb") as f:
    model = pickle.load(f)
model.fit(X_train_tfidf, y_train)

train_class_wise, train_overall = classwise_scores_and_overall(model, X_train_tfidf, y_train)
test_class_wise, test_overall = classwise_scores_and_overall(model,X_test_tfidf, y_test)
oot_class_wise, oot_overall = classwise_scores_and_overall(model,oot_tfidf, y_oot)
train_overall_class_wise, train_overall_overall = classwise_scores_and_overall(model,overall_tfidf, y_train_overall)

classwise = pd.concat([train_class_wise, test_class_wise, train_overall_class_wise, oot_class_wise])
classwise.index = ['train_0', 'train_1', 'train_2', 'test_0', 'test_1', 'test_2',
                   'train_overall_0','train_overall_1','train_overall_2','oot_0', 'oot_1', 'oot_2']

overall = pd.concat([train_overall, test_overall, train_overall_overall, oot_overall])
overall.index = ['train', 'test', 'train_overall','oot']
overall.T

,train,test,train_overall,oot
Overall Accuracy,0.804738,0.795035,0.802312,0.778442
Overall F1 Score,0.795570,0.786121,0.793204,0.760164
Overall Recall,0.804738,0.795035,0.802312,0.778442
Overall Precision,0.797439,0.787702,0.794987,0.767189
Overall AUC,0.910084,0.901564,0.907952,0.911025
Overall Log Loss,0.436617,0.457220,0.441768,0.457033
